# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
import pandas as pd
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.core import Model

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
data_pd = data.to_pandas_dataframe().dropna()
data_pd

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


In [4]:
ws = Workspace.from_config()
experiment_name = 'Hyperparam_experiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [10]:
os.getcwd()

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook195899/code'

In [5]:
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

In [16]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(parameter_space={'penalty':choice('l1', 'l2', 'elasticnet', 'none'), 'C': uniform(0.2,2)})

#TODO: Create your estimator and hyperdrive config
#estimator is deprecated, so I used ScriptRunConfig instead
#estimator = <your estimator here>
src = ScriptRunConfig(source_directory='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook195899/code',
    script='./train.py',
    compute_target='notebook195899'
    ,environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_run_config =HyperDriveConfig(hyperparameter_sampling=param_sampling,primary_metric_name="Accuracy",primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
policy=early_termination_policy,run_config=src, max_total_runs=5)

In [17]:
#TODO: Submit your experiment

hyperdrive_run=experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [18]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [19]:
#TODO: Save the best model
best_run=hyperdrive_run.get_best_run_by_primary_metric()
joblib.dump(best_run.id,'hyperparam_best_model.joblib')

['hyperparam_best_model.joblib']

In [62]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Hyperparam_experiment,HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [23]:
hyperparam_model=hyperdrive_run.register_model(model_name='hyperparam_best_model',model_path='azureml-logs/hyperparam_best_model.joblib')

In [66]:
hyperparam_model

Model(workspace=Workspace.create(name='quick-starts-ws-195899', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-195899'), name=hyperparam_best_model, id=hyperparam_best_model:1, version=1, tags={}, properties={})

In [117]:
hyperparam_model2=best_run.register_model(model_name='hyperparam_best_model2',model_path='azureml-logs/hyperparam_best_model2.joblib')

DEBUG:azureml.Run#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0.RunHistoryFacade.ArtifactsClient:Fetching files for prefix in ExperimentRun, dcid.HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0, azureml-logs/hyperparam_best_model2.joblib
DEBUG:azureml.Run#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0.RunHistoryFacade.ArtifactsClient.list_sas_by_prefix-async:True:[START]
DEBUG:azureml._restclient.service_context.WorkerPool:submitting future: _execute_with_base_arguments
DEBUG:azureml.Run#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0.RunHistoryFacade.ArtifactsClient.list_sas_by_prefix:Using basic handler - no exception handling
DEBUG:azureml.Run#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0.RunHistoryFacade.ArtifactsClient:ClientBase: Calling list_sas_by_prefix with url /artifact/v2.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/artifacts/prefix/contentinfo/{origin}/{container}/{path}
DEBUG:azureml.Run#HD_daaca8e0-23a5

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path azureml-logs/hyperparam_best_model2.joblib in the set of files uploaded to the run: ['azureml-logs/55_azureml-execution-tvmps_4b20a54a508c5f941ed8bbe97fa5d8b289340e7e6d14a9f67a5213a5faaecebd_d.txt', 'azureml-logs/65_job_prep-tvmps_4b20a54a508c5f941ed8bbe97fa5d8b289340e7e6d14a9f67a5213a5faaecebd_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_4b20a54a508c5f941ed8bbe97fa5d8b289340e7e6d14a9f67a5213a5faaecebd_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/93_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided

In [104]:
#model_1_path=Model.get_model_path(model_name='hyperparam_best_model')
model_1=joblib.load('./azureml-models/hyperparam_best_model.joblib')

FileNotFoundError: [Errno 2] No such file or directory: './azureml-models/hyperparam_best_model.joblib'

DEBUG:azureml.HyperDriveRun#HD_45c6d158-91b0-43fb-ad85-eadb60db5618.RunHistoryFacade.RunClient.get-async:False:[START]
DEBUG:azureml.HyperDriveRun#HD_45c6d158-91b0-43fb-ad85-eadb60db5618.RunHistoryFacade.RunClient:ClientBase: Calling get with url /history/v1.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/experiments/{experimentName}/runs/{runId}
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 1800
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urlli

In [120]:
#from azureml.core.model import Model
import logging
logging.basicConfig(level=logging.DEBUG)
print(Model.get_model_path(model_name='hyperparam_best_model.joblib'))

DEBUG:azureml.core.run:Could not load run context RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run."
    }
}, switching offline: False
DEBUG:azureml.core.run:Could not load the run context and allow_offline set to False
DEBUG:azureml.core.model:RunEnvironmentException: RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException RunEnvironmentException:
	Message: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.
	InnerException None
	ErrorResponse 
{
    "error": {
   

hyperparam_best_model.joblib


DEBUG:azureml.HyperDriveRun#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c.RunHistoryFacade.RunClient.get-async:False:[START]
DEBUG:azureml.HyperDriveRun#HD_daaca8e0-23a5-48b6-b749-c4c6257b323c.RunHistoryFacade.RunClient:ClientBase: Calling get with url /history/v1.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/experiments/{experimentName}/runs/{runId}
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 1800
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urlli

In [88]:
model_1

'HD_daaca8e0-23a5-48b6-b749-c4c6257b323c_0'

In [54]:
#model_1.predict(input_data)
hyperparam_model

Model(workspace=Workspace.create(name='quick-starts-ws-195899', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-195899'), name=hyperparam_best_model, id=hyperparam_best_model:1, version=1, tags={}, properties={})

In [105]:
from azureml.core.model import InferenceConfig

#environment = best_run.get_context().get_environment()
inference_config = InferenceConfig(entry_script='entry_script.py', environment=sklearn_env)

DEBUG:azureml.HyperDriveRun#HD_45c6d158-91b0-43fb-ad85-eadb60db5618.RunHistoryFacade.RunClient.get-async:False:[START]
DEBUG:azureml.HyperDriveRun#HD_45c6d158-91b0-43fb-ad85-eadb60db5618.RunHistoryFacade.RunClient:ClientBase: Calling get with url /history/v1.0/subscriptions/{subscriptionId}/resourceGroups/{resourceGroupName}/providers/Microsoft.MachineLearningServices/workspaces/{workspaceName}/experiments/{experimentName}/runs/{runId}
DEBUG:msrest.service_client:Accept header absent and forced to application/json
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): 127.0.1.1:46808
DEBUG:urllib3.connectionpool:http://127.0.1.1:46808 "GET /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 HTTP/1.1" 200 1800
DEBUG:msrestazure.azure_active_directory:MSI: Retrieving a token from http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01
DEBUG:msrestazure.azure_active_directory:MSI: token retrieved
DEBUG:urlli

In [59]:
service=Model.deploy(ws, 'hyperparam-webservice5',[hyperparam_model], inference_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-05-17 11:56:09+00:00 Creating Container Registry if not exists.
2022-05-17 11:56:09+00:00 Registering the environment.
2022-05-17 11:56:10+00:00 Use the existing image.
2022-05-17 11:56:10+00:00 Generating deployment configuration.
2022-05-17 11:56:11+00:00 Submitting deployment to compute.
2022-05-17 11:56:14+00:00 Checking the status of deployment hyperparam-webservice4..
2022-05-17 12:00:17+00:00 Checking the status of inference endpoint hyperparam-webservice4.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e86a2ccb-6d77-42d7-922f-b59c3a520954
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: hyperparam-webservice4. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 30309f9096e748bdb3332a4da8cc082b.azurecr.io/azureml/azureml_14456edd520c7b69e8bdf340b7bbe4c1 locally. Please refer to https://aka.ms/debugimage#service-la

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e86a2ccb-6d77-42d7-922f-b59c3a520954
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: hyperparam-webservice4. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image 30309f9096e748bdb3332a4da8cc082b.azurecr.io/azureml/azureml_14456edd520c7b69e8bdf340b7bbe4c1 locally. 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [27]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = 'http://409a7c3d-dc06-4851-8946-d6ef1f66bee2.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
#key = ''

# Two sets of data to score, so we get two results back
data = {"data":
        [
          {
            "age": 17,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 971,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
          {
            "age": 87,
            "campaign": 1,
            "cons.conf.idx": -46.2,
            "cons.price.idx": 92.893,
            "contact": "cellular",
            "day_of_week": "mon",
            "default": "no",
            "duration": 471,
            "education": "university.degree",
            "emp.var.rate": -1.8,
            "euribor3m": 1.299,
            "housing": "yes",
            "job": "blue-collar",
            "loan": "yes",
            "marital": "married",
            "month": "may",
            "nr.employed": 5099.1,
            "pdays": 999,
            "poutcome": "failure",
            "previous": 1
          },
      ]
    }
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
#headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data)#, headers=headers)
print(resp.json())


test is {'data': [{'age': 17, 'campaign': 1, 'cons.conf.idx': -46.2, 'cons.price.idx': 92.893, 'contact': 'cellular', 'day_of_week': 'mon', 'default': 'no', 'duration': 971, 'education': 'university.degree', 'emp.var.rate': -1.8, 'euribor3m': 1.299, 'housing': 'yes', 'job': 'blue-collar', 'loan': 'yes', 'marital': 'married', 'month': 'may', 'nr.employed': 5099.1, 'pdays': 999, 'poutcome': 'failure', 'previous': 1}, {'age': 87, 'campaign': 1, 'cons.conf.idx': -46.2, 'cons.price.idx': 92.893, 'contact': 'cellular', 'day_of_week': 'mon', 'default': 'no', 'duration': 471, 'education': 'university.degree', 'emp.var.rate': -1.8, 'euribor3m': 1.299, 'housing': 'yes', 'job': 'blue-collar', 'loan': 'yes', 'marital': 'married', 'month': 'may', 'nr.employed': 5099.1, 'pdays': 999, 'poutcome': 'failure', 'previous': 1}]}


In [ ]:
scoring_uri = service.scoring_uri
resp = requests.post(scoring_uri, input_data)#, headers=headers)
print(resp.json())

In [34]:
test_sample = json.load(open('data.json'))

prediction = service.run(test_sample)

print(prediction)


JSONDecodeError: [Errno Expecting value] name 'requests' is not defined: 0

In [35]:
scoring_uri = 'http://409a7c3d-dc06-4851-8946-d6ef1f66bee2.southcentralus.azurecontainer.io/score'
resp = requests.post(scoring_uri, input_data)#, headers=headers)
print(resp.json())

test is {'data': [{'age': 17, 'campaign': 1, 'cons.conf.idx': -46.2, 'cons.price.idx': 92.893, 'contact': 'cellular', 'day_of_week': 'mon', 'default': 'no', 'duration': 971, 'education': 'university.degree', 'emp.var.rate': -1.8, 'euribor3m': 1.299, 'housing': 'yes', 'job': 'blue-collar', 'loan': 'yes', 'marital': 'married', 'month': 'may', 'nr.employed': 5099.1, 'pdays': 999, 'poutcome': 'failure', 'previous': 1}, {'age': 87, 'campaign': 1, 'cons.conf.idx': -46.2, 'cons.price.idx': 92.893, 'contact': 'cellular', 'day_of_week': 'mon', 'default': 'no', 'duration': 471, 'education': 'university.degree', 'emp.var.rate': -1.8, 'euribor3m': 1.299, 'housing': 'yes', 'job': 'blue-collar', 'loan': 'yes', 'marital': 'married', 'month': 'may', 'nr.employed': 5099.1, 'pdays': 999, 'poutcome': 'failure', 'previous': 1}]}


TODO: In the cell below, print the logs of the web service and delete the service

In [60]:
print(service.get_logs())

2022-05-17T12:05:25,608029910+00:00 - iot-server/run 
2022-05-17T12:05:25,910077407+00:00 - nginx/run 
2022-05-17T12:05:26,013141721+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-05-17T12:05:26,109487494+00:00 - rsyslog/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-05-17T12:05:28,605269847+00:00 - iot-server/finish 1 0
2022-05-17T12:05:28,607632761+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (13)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 38
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2022-05-17 12:05:48,304 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2022-05-17 12:05:48,305 | root | INFO | Starting up request id generator
2022-05-17 12:05:48,305 | root | INFO | Star

In [ ]:
service.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

